<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Bitcoin/Bitcoin_Identification_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

# Téléchargement des données

In [ ]:
!wget --no-check-certificate --content-disposition "https://github.com/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Bitcoin/Bitcoin_complet.zip?raw=true"
!unzip Bitcoin_complet.zip

In [ ]:
df_data=pd.read_csv('Bitcoin_complet.csv')
df_data.shape

In [ ]:
df_data.head()

# Préparation des jeux de données

In [ ]:
# Déplace la colonne Prix au début
col_prix = df_data.pop('Price')
df_data.insert(1,'Price',col_prix)
df_data.head()

In [ ]:
X_raw = df_data.iloc[:,2:20]
X_raw.head()

In [ ]:
y = df_data.iloc[:,1:2]
y.head()

# Application de l'algorithme Random Forest

In [ ]:
tf.squeeze(np.asarray(y),1)

In [ ]:
features_list=[]
indicateurs_techniques=['sma','ema','wma','trix', 'std','var','rsi','roc']
periode=['3','7','14','30','90']

for indicateur in indicateurs_techniques:
    for i in periode:
      print("%s%s" %(indicateur, i))
      filtre = str(indicateur) + str(i) + "$"

      X = df_data.filter(regex=filtre,axis=1)
      X = SimpleImputer(missing_values=np.nan,strategy='most_frequent').fit_transform(X)
      X = pd.DataFrame(X)
      X.columns = df_data.filter(regex=filtre,axis=1).columns

      rf1 = RandomForestRegressor(random_state=7,n_jobs=-1)
      rfecv = RFECV(rf1,step=0.9,min_features_to_select=1,verbose=1,cv=5,scoring='neg_mean_absolute_error', n_jobs=1)
      rfecv.fit(X,tf.squeeze(np.asarray(y),1))

      if rfecv.n_features_ > 1:
        rf1.fit(X,tf.squeeze(np.asarray(y),1))
        maximp = rf1.feature_importances_.max()
        for x in range(len(rf1.feature_importances_)):
          if maximp==rf1.feature_importances_[x]:
            new_features = X.columns[x]
      else:
        mask = rfecv.get_support()
        new_features = X.columns[mask]
      features_list.append(str(new_features))
      print(filtre+ ': ' + new_features)

In [ ]:
X